In [5]:
import csv
import requests
from bs4 import BeautifulSoup

# Define a function to extract the required information from each link
def extract_property_data(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Extract price
    price_meta = soup.find('meta', itemprop='price')
    price = price_meta['content'] if price_meta else None
    
    # Extract property name
    name_span = soup.find('span', class_='ib')
    name = name_span.find('span', itemprop='name').text if name_span else None
    
    # Extract latitude and longitude
    geo_div = soup.find('div', itemprop='geo')
    latitude = geo_div.find('meta', itemprop='latitude')['content'] if geo_div else None
    longitude = geo_div.find('meta', itemprop='longitude')['content'] if geo_div else None
    
    # Extract status
    status_td = soup.find('td', id='Status')
    status = status_td.get('title') if status_td else None
    
    # Extract rate per square feet
    rate_wrap = soup.find('div', class_='rate-wrap')
    rate = rate_wrap.text.strip() if rate_wrap else None
    
    # Extract age of property
    age_td = soup.find('td', id='Age of Property', class_='val')
    age = age_td.get('title') if age_td else None
    
    return {
        'Price': price,
        'Name': name,
        'Latitude': latitude,
        'Longitude': longitude,
        'Status': status,
        'Rate per Sq Ft': rate,
        'Age of Property': age
    }

# Read the input CSV file and write the output CSV file
with open('delhi_property_listings.csv', mode='r', newline='', encoding='utf-8') as infile, \
     open('delhi_extracted_property_data.csv', mode='w', newline='', encoding='utf-8') as outfile:
    
    reader = csv.DictReader(infile)
    fieldnames = ['Price', 'Name', 'Latitude', 'Longitude', 'Status', 'Rate per Sq Ft', 'Age of Property']
    writer = csv.DictWriter(outfile, fieldnames=fieldnames)
    
    writer.writeheader()
    
    for row in reader:
        url = row['Listing Href Link']
        data = extract_property_data(url)
        writer.writerow(data)
